# Live Setup

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from gtts import gTTS
import os

model = load_model('residual_cnn_model.keras')

# Class names for the ASL letters
classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']

cap = cv2.VideoCapture(0)

# List to store the sequence of letters
text_sequence = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    
    roi = frame[100:300, 100:300]
    
    # Preprocess the ROI
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    normalized = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX)
    resized = cv2.resize(normalized, (28, 28))
    reshaped = np.reshape(resized, (1, 28, 28, 1)) / 255.0
    
    # Predict the letter
    result = model.predict(reshaped)
    index = np.argmax(result)
    
    if index < len(classes):
        letter = classes[index]
    else:
        letter = '?'
    
    # Display the predicted letter on the screen
    cv2.putText(frame, letter, (100, 90), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2)
    cv2.rectangle(frame, (100, 100), (300, 300), (0, 255, 0), 2)
    cv2.imshow('Frame', frame)
    
    # Save the letter if 's' is pressed, exit if 'q' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('s'):
        text_sequence.append(letter)
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Convert the resulting text sequence to speech
text = ''.join(text_sequence)
print("Predicted text:", text)

# Use gTTS to convert text to speech
if text:
    tts = gTTS(text=text, lang='en')
    tts.save("predicted_text.mp3")
    os.system("start predicted_text.mp3")  # For Windows, use "start", for macOS use "open", and for Linux use "xdg-open"
